In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/Kaludi/Customer-Support-Responses/Customer-Support.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
df.head()

,query,response
0,My order hasn't arrived yet.,We apologize for the inconvenience. Can you pl...
1,I received a damaged product.,We apologize for the inconvenience. Can you pl...
2,I need to return an item.,Certainly. Please provide your order number an...
3,I want to change my shipping address.,No problem. Can you please provide your order ...
4,I have a question about my bill.,We'd be happy to help. Can you please provide ...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   query     74 non-null     object
 1   response  74 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [4]:
df.describe()

,query,response
count,74,74
unique,74,73
top,My order hasn't arrived yet.,Thank you for your interest. Can you please pr...
freq,1,2


In [5]:
df.isnull().sum()

query       0
response    0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2)

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorForSeq2Seq

class CustomerSupportDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        input_text = item['query']
        target_text = item['response']

        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        labels = target_encoding["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': encoding["input_ids"].flatten(),
            'attention_mask': encoding["attention_mask"].flatten(),
            'labels': labels.flatten()
        }

# Create datasets
train_dataset = CustomerSupportDataset(train_df, tokenizer)
val_dataset = CustomerSupportDataset(val_df, tokenizer)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [9]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,No log,1.593491
2,No log,1.352615


Epoch,Training Loss,Validation Loss
1,No log,1.593491
2,No log,1.352615
3,No log,1.319928


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=45, training_loss=2.005756293402778, metrics={'train_runtime': 2173.5811, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.021, 'total_flos': 53961676554240.0, 'train_loss': 2.005756293402778, 'epoch': 3.0})

In [10]:
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt', max_length=max_length, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "How can I reset my password?"
response = generate_response(prompt, model, tokenizer)
print(response)

We'd be happy to help. Can you please provide your password so we can reset it?


In [13]:
# Save the trained model
trainer.save_model('./saved')

# Save the tokenizer
tokenizer.save_pretrained('./saved')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./saved/tokenizer_config.json',
 './saved/special_tokens_map.json',
 './saved/vocab.json',
 './saved/merges.txt',
 './saved/added_tokens.json')

In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained('./saved')

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('./saved')


In [17]:
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors='pt', max_length=max_length, truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "How can I reset my password"
response = generate_response(prompt, model, tokenizer)
print(response)

We'd be happy to help. Can you please provide your password so we can reset it?
